In [109]:
import csv
import sys, os
import datetime
import dateutil.parser

# open NOTEEVENTS.csv to extract discharge summaries
d = open("../../mimic/NOTEEVENTS.csv")
reader = csv.reader(d)
header = next(reader)

# open ADMISSIONS.csv to extract discharge summaries
a = open("../../mimic/ADMISSIONS.csv")
reader2 = csv.reader(a)
header2 = next(reader2)

In [92]:
summaries = {}
for row in reader:
    if row[6] == 'Discharge summary':
        summaries[row[2]] = row[1]

In [99]:
readmit = {}
for row in reader2:
    if row[2] in readmit:
        readmit[row[2]].append(row[1])
        readmit[row[2]].append(row[3])
        readmit[row[2]].append(row[4])
    else:
        readmit[row[2]] = []
        readmit[row[2]].append(row[1])
        readmit[row[2]].append(row[3])
        readmit[row[2]].append(row[4])

In [19]:
labels = {}
for row in reader:
    if row[1] in labels:
        labels[row[1]] += 1
    else:
        labels[row[1]] = 1

In [88]:
with open('../../data/update/filtered.txt','r') as f:
    patients = f.readlines()
    filtered = []
    for i in range(len(patients)):
        filtered.append(patients[i].strip('\n'))

set_filtered = set(filtered)

In [100]:
# filtering out the discharge summaries of dead people and babies
filtered_summaries = {}
for key in summaries:
    if summaries[key] not in set_filtered:
        filtered_summaries[key] = summaries[key]

# find how many admissions do not have a discharge summary
missing = {}
existing_summaries = {}
for key in readmit:
    if key not in filtered_summaries:
        missing[key] = readmit[key][0]
    else:
        existing_summaries[key] = filtered_summaries[key]
        
# of the missing discharge summaries, see which match with a filtered patient
filtered_missing = {}
for key in missing:
    if missing[key] not in set_filtered:
        filtered_missing[key] = missing[key]

# check how many existing admissions there are after filtering
filtered_readmit = {}
for key in readmit:
    if readmit[key][0] not in set_filtered:
        if key not in filtered_missing:
            filtered_readmit[key] = readmit[key]

In [156]:
with open('../../data/update/splits/filtered_readmit','w') as f:
    for key in filtered_readmit:
        f.write(key)
        f.write('\n')

In [110]:
# generate the admit and discharge times for every admission with a discharge summary
# key is patient ID
# value is a list of the admission IDs, admit times, discharge times
times = {}
for row in reader2:
    if row[2] in filtered_readmit:
        if row[1] in times:
            times[row[1]].append(row[2])
            times[row[1]].append(row[3])
            times[row[1]].append(row[4])
        else:
            times[row[1]] = []
            times[row[1]].append(row[2])
            times[row[1]].append(row[3])
            times[row[1]].append(row[4])

In [101]:
len(filtered_readmit)

41176

In [102]:
len(existing_summaries)

41176

In [42]:
def check_readmit(times):
    readmit_label = []
    if len(times)>2:
        i=2
        while i < len(times)-1:
            discharge = dateutil.parser.parse(times[i])
            readmit = dateutil.parser.parse(times[(i+2)])
            readmit_cutoff = discharge + datetime.timedelta(days=30)
            if readmit < readmit_cutoff:
                print('30 day readmit')
                readmit_label.append([1,times[i-2]])
            else:
                print('over 30 days')
                readmit_label.append([0,times[i-2]])
            i += 3
        readmit_label.append([0,times[i-2]])
    else:
        print ('no readmit')
        readmit_label.append([0,times[0]])
    
    return readmit_label

In [150]:
def write_label(readmit_label,patient_ID):
    for i in range(len(readmit_label)):
        adm_ID = readmit_label[i][1]
        path = patient_ID + '_' + adm_ID
        label = readmit_label[i][0]
        with open('../../data/update/processPatient/'+ path,'r') as f:
            with open('../../data/update/labels/'+ path,'w') as g:
                lines = len(f.readlines())
                for i in range(lines):
                    if i == (lines-1):
                        if label == 1:
                            g.write('1')
                        else:
                            g.write('0')
                    else:
                        if label == 1:
                            g.write('1\n')
                        else:
                            g.write('0\n')

In [140]:
admissions = os.listdir('../../data/update/processPatient/')
patientID = []
for i in range(len(admissions)):
    patientID.append(admissions[i].split('_'))

patients = []
for i in range(len(patientID)):
    patients.append(patientID[i][0])

patient_set = set(patients)
patient_list = list(patient_set)

In [142]:
len(patient_list)

32120

In [151]:
for i in range(len(patient_list)):
    test_patient = patient_list[i]
    test_readmit = check_readmit(times[test_patient])
    write_label(test_readmit, test_patient)

over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 

over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 

30 day readmit
30 day readmit
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
30 day readmit
30 day read

30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over

over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
o

...

over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
30 day

over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 

over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days


over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 da

30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 day

In [14]:
readmit_count = 0
for i in patientList:
    if labels[i]>1:
        readmit_count += 1
        

In [16]:
readmit_count

5834

In [17]:
len(patientList)

32120

In [24]:
readmit['7101']

['173085',
 '2157-09-24 10:05:00',
 '2157-10-14 19:30:00',
 '161534',
 '2160-02-25 18:43:00',
 '2160-03-01 15:51:00',
 '161140',
 '2162-04-03 00:32:00',
 '2162-04-09 18:25:00',
 '177536',
 '2162-04-17 23:00:00',
 '2162-05-07 16:50:00']

In [145]:
check_date(times['7101'])

over 30 days
over 30 days
30 day readmit


[[0, '173085'], [0, '161534'], [1, '161140'], [0, '177536']]

In [45]:
test_patient = patientID[20][0]
test_readmit = check_readmit(readmit[test_patient])
write_label(test_readmit, test_patient)

In [54]:
check_readmit(readmit['23657'])

over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit
30 day readmit
over 30 days
30 day readmit
over 30 days
30 day readmit
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
over 30 days
30 day readmit


[[0, '192260'],
 [0, '103090'],
 [0, '161274'],
 [0, '134743'],
 [0, '164590'],
 [1, '125544'],
 [1, '169263'],
 [0, '176997'],
 [1, '157017'],
 [0, '156416'],
 [1, '102662'],
 [0, '137894'],
 [0, '148447'],
 [0, '118450'],
 [0, '121592'],
 [0, '155617'],
 [0, '174141'],
 [0, '177550'],
 [1, '130222'],
 [0, '113800']]

In [81]:
readmit['26013']

['155657',
 '2108-05-25 22:55:00',
 '2108-06-11 13:01:00',
 '137187',
 '2111-08-29 16:08:00',
 '2111-09-30 18:30:00',
 '120646',
 '2112-05-10 20:53:00',
 '2112-05-19 16:10:00']

In [72]:
len(readmit)

32897

In [152]:
label_path = os.listdir('../../data/update/labels/')
positive = 0
negative = 0
for i in range(len(label_path)):
    with open('../../data/update/labels/'+label_path[i],'r') as f:
        test = f.readline()
        if test == '1' or test == '1\n':
            positive += 1
        else:
            negative += 1

In [153]:
print(positive)
print(negative)

2324
38852


In [154]:
len(filtered_missing)

1317